CONEXIÓN CON STORAGE

In [0]:
%scala

val storageAccount = "sandovalstorage"
val container = "proyectofinal"
val sasKey = "?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-03-03T09:24:59Z&st=2024-02-03T01:24:59Z&spr=https&sig=SfM79JEEiOFinkOvEfMsrE1Xd09QFoBNMvYw0d%2B3RiM%3D"

val conf = "fs.azure.sas." + container + "." + storageAccount + ".blob.core.windows.net"

dbutils.fs.mount(source="wasbs://" + container + "@" + storageAccount + ".blob.core.windows.net", mountPoint="/mnt/" + container, extraConfigs = Map(conf -> sasKey))

storageAccount: String = sandovalstorage
container: String = proyectofinal
sasKey: String = ?sv=2022-11-02&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2024-03-03T09:24:59Z&st=2024-02-03T01:24:59Z&spr=https&sig=SfM79JEEiOFinkOvEfMsrE1Xd09QFoBNMvYw0d%2B3RiM%3D
conf: String = fs.azure.sas.proyectofinal.sandovalstorage.blob.core.windows.net
res0: Boolean = true

In [0]:
##Lee información de la capa Plata en un dataframe
dfProducto = spark.read.parquet("/mnt/proyectofinal/Capa Plata/Producto.parquet")
dfCategoria = spark.read.parquet("/mnt/proyectofinal/Capa Plata/Categoria.parquet")
dfSubCategoria = spark.read.parquet("/mnt/proyectofinal/Capa Plata/SubCategoria.parquet")

In [0]:
##Creamos vista temporal con cada dataframe
dfProducto.createOrReplaceTempView("Producto")
dfProducto.createOrReplaceTempView("Categoria")
dfProducto.createOrReplaceTempView("SubCategoria")

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW Fact_Producto AS 
SELECT P.Cod_Producto, P.Nombre AS NombreProducto, S.Nombre AS NombreSubCategoria, C.Nombre AS NombreCategoria
FROM Producto AS P
JOIN SubCategoria AS S
ON P.Cod_SubCategoria = S.Cod_SubCategoria
JOIN Categoria AS C
ON S.Cod_SubCategoria = C.Cod_SubCategoria

In [0]:
dfFactProducto = spark.table("Fact_Producto")
dfFactProducto.write.mode("overwrite").parquet("/mnt/proyectofinal/Capa Oro/FacProducto.parquet")

In [0]:
dbutils.fs.unmount("/mnt/proyectofinal")

/mnt/proyectofinal has been unmounted.


True